# Churned Customer Study Notebook

## Objectives

*   Answer business requirement 1: 
    * The client is interested to understand the patterns from the customer base, so the client can learn the most relevant variables that are correlated to a churned customer.

## Inputs

* outputs/datasets/collection/TelcoCustomerChurn.csv

## Outputs

* generate code that answers business requirement 1 and can be used to build Streamlit App






---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/TelcoCustomerChurn.csv")
    .drop(['customerID'], axis=1)
    )
df.head(3)

# Data Inspection

We are interested to get more familiar with the dataset, check variable type and distribution, missing levels and what these variables mean in a business context

In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

# Correlation Study

For this analysis, we are transforming the categorical variables with One Hot Encoding technique, so we can see which levels tend to affect more Churn Levels

In [ ]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=df.columns[df.dtypes=='object'].to_list(), drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
df_ohe.head(3)

We use `.corr()` for `spearman` and `pearson` methods, and investigate the top 10 correlations
* We know this command returns a pandas series and the first item is the correlation between Churn and Churn, which happens to be 1, so we exclude that with `[1:]`
* We sort values considering the aboslute value, by setting `key=abs`

In [ ]:
corr_spearman = df_ohe.corr(method='spearman')['Churn'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_spearman

We do the same for `pearson`

In [ ]:
corr_pearson = df_ohe.corr(method='pearson')['Churn'].sort_values(key=abs, ascending=False)[1:].head(10)
corr_pearson

For both methods, we notice weak or moderate levels of correlation between Churn and a given variable. 
* Ideally, we pursue at strong correlation levels or more. However, this is not always possible

We will consider the top 7 correlation levels at `df_ohe` and will study the associated variables at `df`

In [ ]:
top_n = 5
set(corr_pearson[:top_n].index.to_list() + corr_spearman[:top_n].index.to_list())

Therefore we are studying at df the following variables. We will investigate if:
* A churned customer typically has a month to month contract
* A churned customer typically has fiber optic
* A churned customer typically doesn't have tech support
* A churned customer doesn't have online security
* A churned customer typically has low tenure levels

In [ ]:
vars_to_study = ['Contract', 'InternetService', 'OnlineSecurity', 'TechSupport', 'tenure']
vars_to_study

# EDA on selected variables

In [ ]:
df_eda = df.filter(vars_to_study + ['Churn'])
df_eda.head(3)

## Variables Distribution by Churn

We plot the distribution (numerical and categorical) colored by Churn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

def plot_categorical(df, col, target_var):

  plt.figure(figsize=(12, 5))
  sns.countplot(data=df, x=col, hue=target_var,order = df[col].value_counts().index)
  plt.xticks(rotation=90) 
  plt.title(f"{col}", fontsize=20,y=1.05)        
  plt.show()

def plot_numerical(df, col, target_var):
  plt.figure(figsize=(8, 5))
  sns.histplot(data=df, x=col, hue=target_var, kde=True,element="step") 
  plt.title(f"{col}", fontsize=20,y=1.05)
  plt.show()



target_var = 'Churn'
for col in vars_to_study:
  if df_eda[col].dtype == 'object':
    plot_categorical(df_eda, col, target_var)
    print("\n\n")
  else:
    plot_numerical(df_eda, col, target_var)
    print("\n\n")

---

## Parallel Plot

Create a separate DataFrame and transform `tenure` (numerical) into bins (categorical) for visualizing at `parallel_categories()` plot


In [ ]:
from feature_engine.discretisation import ArbitraryDiscretiser
import numpy as np
tenure_map = [-np.Inf, 6, 12, 18, 24, np.Inf]
disc = ArbitraryDiscretiser(binning_dict={'tenure': tenure_map})
df_parallel = disc.fit_transform(df_eda)
df_parallel.head()

In [ ]:
disc.binner_dict_['tenure']

Create map to replace `tenure` variable with more informative levels

In [ ]:
n_classes = len(tenure_map) - 1
classes_ranges = disc.binner_dict_['tenure'][1:-1]

labels_map = {}
for n in range(0,n_classes):
  if n == 0:
    labels_map[n] = f"<{classes_ranges[0]}"
  elif n == n_classes-1:
    labels_map[n] = f"+{classes_ranges[-1]}"
  else:
    labels_map[n] = f"{classes_ranges[n-1]} to {classes_ranges[n]}"

labels_map

Replace using `.replace()`

In [ ]:
df_parallel['tenure'] = df_parallel['tenure'].replace(labels_map)
df_parallel.head()

Creates multi-dimensional categorical data plot

In [ ]:
import plotly.express as px
fig = px.parallel_categories(df_parallel, color="Churn")
fig.show()

---

# Conclusions and Next steps

The correlations and plots interpretation converge. 
* A churned customer typically has a month to month contract
* A churned customer typically has fiber optic
* A churned customer typically doesn't have tech support
* A churned customer doesn't have online security
* A churned customer typically has low tenure levels

---

Well done! You generated code that answers business requirement 1 and can be used to build Streamlit App
* In this notebook we are not pushing any file to the repo
* Clear cell's outputs before moving on!